# 1. What is a neural network?

# 2. How do you check the performance?

# 3. Data Cleaning

In [44]:
import pandas as pd
import numpy as np

adult_df = pd.read_csv('adult.csv')
adult_df.columns = ['age','workclass','finalweight',
                        'education','education_num',
                        'marital_status','occupation',
                        'relationship','race',
                        'sex','capital_gain',
                        'capital_loss',
                        'hours_per_week',
                        'native_country',
                        'label']
adult_df.columns

adult_df.head()

,age,workclass,finalweight,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,label
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [45]:
adult_df.dtypes

age                int64
workclass         object
finalweight        int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
label             object
dtype: object

In [46]:
def preprocessing():
    adult_df = pd.read_csv('adult.csv')
    adult_df.columns = ['age','workclass','finalweight',
                        'education','education_num',
                        'marital_status','occupation',
                        'relationship','race',
                        'sex','capital_gain',
                        'capital_loss',
                        'hours_per_week',
                        'native_country',
                        'label']
    
    workclass = pd.get_dummies(adult_df['workclass'])
    adult_df = pd.concat([adult_df, workclass], axis=1)
    adult_df = adult_df.drop(['workclass'], axis=1)
    
    # i can drop education because the df also has education num, which
    # is just education but made into numerical values
    adult_df = adult_df.drop(['education'], axis=1)
    
    marital_status = pd.get_dummies(adult_df['marital_status'])
    adult_df = pd.concat([adult_df, marital_status], axis=1)
    adult_df = adult_df.drop(['marital_status'], axis=1)
    
    occupation = pd.get_dummies(adult_df['occupation'])
    adult_df = pd.concat([adult_df, occupation], axis=1)
    adult_df = adult_df.drop(['occupation'], axis=1)
    
    relationship = pd.get_dummies(adult_df['relationship'])
    adult_df = pd.concat([adult_df, relationship], axis=1)
    adult_df = adult_df.drop(['relationship'], axis=1)
    
    race = pd.get_dummies(adult_df['race'])
    adult_df = pd.concat([adult_df, race], axis=1)
    adult_df = adult_df.drop(['race'], axis=1)
    
    sex = pd.get_dummies(adult_df['sex'])
    adult_df = pd.concat([adult_df, sex], axis=1)
    adult_df = adult_df.drop(['sex'], axis=1)
    
    native_country = pd.get_dummies(adult_df['native_country'])
    adult_df = pd.concat([adult_df, native_country], axis=1)
    adult_df = adult_df.drop(['native_country'], axis=1)
    
    adult_df['label'] = adult_df['label'].map({' <=50K': 0, ' >50K': 1})
        
    return adult_df

In [47]:
# adult_df = preprocessing()
adult_df = preprocessing()

In [52]:
pd.set_option('display.max_rows', 92)


In [53]:
adult_df.dtypes

age                 int64
finalweight         int64
education_num       int64
capital_gain        int64
capital_loss        int64
                    ...  
 Thailand            bool
 Trinadad&Tobago     bool
 United-States       bool
 Vietnam             bool
 Yugoslavia          bool
Length: 93, dtype: object

In [49]:
adult_df['label'].value_counts()

label
0    24719
1     7841
Name: count, dtype: int64

# 4. Neural Network

In [50]:
import torch
from sklearn.model_selection import train_test_split

X = adult_df.drop('label', axis=1).values
y = adult_df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=13,
                                                    stratify=y)


In [51]:
import torch.nn as nn
import torch.nn.functional as F

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.